In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
# Load the ferry trips data
ferry_data = pd.read_csv('data/ferry_trips_data_large.csv')

# Display the first few rows to understand the structure
ferry_data.head()
# Summary of the dataset
#ferry_data.info()

,route_id,route_name,ferry_name,ferry_id,terminal_departure,terminal_arrival,time_departure,cars_outbound,trucks_outbound,trucks_with_trailer_outbound,...,passenger_car_equivalent_outbound,passenger_car_equivalent_inbound,fuelcons_outbound_l,distance_outbound_nm,start_time_outbound,end_time_outbound,fuelcons_inbound_l,distance_inbound_nm,start_time_inbound,end_time_inbound
0,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-03-01 05:44:00,6,0,0,...,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-03-01 06:04:00,4,0,0,...,4.0,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-03-01 06:24:00,8,1,0,...,10.5,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-03-01 06:44:00,5,1,0,...,7.5,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-03-01 07:04:00,7,0,0,...,7.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Q1 What is the total fuel cost for ferry Jupiter in January 2024 assuming that the price per liter is 20 sek?

In [26]:
# Convert 'time_departure' to datetime objects for filtering
ferry_data['time_departure'] = pd.to_datetime(ferry_data['time_departure'])

# Filter data for ferry 'Jupiter' and January 2024
jupiter_jan_data = ferry_data[
    (ferry_data['ferry_name'] == 'Jupiter') &
    (ferry_data['time_departure'].dt.month == 1) &
    (ferry_data['time_departure'].dt.year == 2024)
]

# Calculate total fuel consumption (handling potential NaNs by filling with 0)
total_fuel_consumption = (
    jupiter_jan_data['fuelcons_outbound_l'].fillna(0).sum() +
    jupiter_jan_data['fuelcons_inbound_l'].fillna(0).sum()
)

# Assuming a fuel cost per liter (you might need to adjust this)
# Let's assume a fuel cost of 10 SEK per liter for example purposes
fuel_cost_per_liter = 20

# Calculate the total fuel cost
total_fuel_cost = total_fuel_consumption * fuel_cost_per_liter

print(f"Total fuel consumption for ferry Jupiter in January 2024: {total_fuel_consumption:.2f} liters")
print(f"Total fuel cost for ferry Jupiter in January 2024: {total_fuel_cost:.2f} SEK")

Total fuel consumption for ferry Jupiter in January 2024: 12709.34 liters
Total fuel cost for ferry Jupiter in January 2024: 254186.70 SEK


## Q2 What is the average speed of ferry Jupiter? (in km/h)


In [27]:
# Convert time columns to datetime objects
ferry_data['start_time_outbound'] = pd.to_datetime(ferry_data['start_time_outbound'])
ferry_data['end_time_outbound'] = pd.to_datetime(ferry_data['end_time_outbound'])
ferry_data['start_time_inbound'] = pd.to_datetime(ferry_data['start_time_inbound'])
ferry_data['end_time_inbound'] = pd.to_datetime(ferry_data['end_time_inbound'])

# Filter data for ferry 'Jupiter'
jupiter_data = ferry_data[ferry_data['ferry_name'] == 'Jupiter'].copy()

# Function to calculate trip duration in hours, handling potential NaNs
def calculate_duration(start_time, end_time):
    if pd.notna(start_time) and pd.notna(end_time):
        duration = (end_time - start_time).total_seconds() / 3600
        return duration
    return None

# Apply the function to calculate outbound and inbound durations
jupiter_data['duration_outbound'] = jupiter_data.apply(
    lambda row: calculate_duration(row['start_time_outbound'], row['end_time_outbound']), axis=1
)
jupiter_data['duration_inbound'] = jupiter_data.apply(
    lambda row: calculate_duration(row['start_time_inbound'], row['end_time_inbound']), axis=1
)

# Calculate average speed for outbound trips (distance in nautical miles, duration in hours)
jupiter_data['speed_outbound_knots'] = jupiter_data.apply(
    lambda row: row['distance_outbound_nm'] / row['duration_outbound']
    if row['duration_outbound'] else None,
    axis=1
)

# Calculate average speed for inbound trips
jupiter_data['speed_inbound_knots'] = jupiter_data.apply(
    lambda row: row['distance_inbound_nm'] / row['duration_inbound']
    if row['duration_inbound'] else None,
    axis=1
)

# Calculate the average of all valid speed values
valid_speeds = pd.concat([jupiter_data['speed_outbound_knots'].dropna(), jupiter_data['speed_inbound_knots'].dropna()])

if not valid_speeds.empty:
    average_speed_knots = valid_speeds.mean()
    print(f"The average speed of ferry Jupiter is: {average_speed_knots:.2f} knots")

    # Convert knots to km/h and mph for better understanding
    average_speed_kmh = average_speed_knots * 1.852
    average_speed_mph = average_speed_knots * 1.15078
    print(f"Average speed in km/h: {average_speed_kmh:.2f} km/h")
else:
    print("Could not calculate the average speed for ferry Jupiter due to missing distance or time information.")

The average speed of ferry Jupiter is: 6.23 knots
Average speed in km/h: 11.55 km/h


## Q3 How many vehicles are on average left at the terminal for inbound trips? 

In [28]:
import pandas as pd

def calculate_avg_vehicles_left_inbound(df):
    """
    Calculates the average number of vehicles left at the terminal for inbound trips.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        float: average number of vehicles left at terminal for inbound trips
    """
    # Calculate average vehicles left at terminal for inbound trips only
    avg_vehicles_left_inbound = df['vehicles_left_at_terminal_inbound'].mean()
    
    # Print the results
    print("Analysis of Vehicles Left at Terminal for Inbound Trips:")
    print(f"Average vehicles left at terminal per inbound trip: {avg_vehicles_left_inbound:.4f}")
    
    # Format the answer for Q3
    print(f"\nQ3 Answer: On average, {avg_vehicles_left_inbound:.4f} vehicles are left at the terminal for inbound trips.")
    
    return avg_vehicles_left_inbound

# Load the ferry trips data
try:

    # Execute the function
    calculate_avg_vehicles_left_inbound(ferry_data)
    
except Exception as e:
    print(f"Error: {e}") 

Analysis of Vehicles Left at Terminal for Inbound Trips:
Average vehicles left at terminal per inbound trip: 0.3270

Q3 Answer: On average, 0.3270 vehicles are left at the terminal for inbound trips.


## Q4 How many vehicles are on average unable to board (left behind at the terminal) per trip for ferry Jupiter during summer months (6, 7, 8)?

In [29]:
# No need to import pandas or load data again - ferry_data is already available

# Convert 'time_departure' to datetime objects for filtering (if not already done)
ferry_data['time_departure'] = pd.to_datetime(ferry_data['time_departure'], errors='coerce')

# Filter data for ferry 'Jupiter'
jupiter_data = ferry_data[ferry_data['ferry_name'] == 'Jupiter'].copy()

# Define summer months (6, 7, 8)
summer_months = [6, 7, 8]

# Filter Jupiter data for summer months, handling potential NaT values
jupiter_summer_data = jupiter_data[
    jupiter_data['time_departure'].notna() &
    jupiter_data['time_departure'].dt.month.isin(summer_months)
]

# Calculate the total vehicles LEFT at the terminal (not transported) during summer for Jupiter
left_at_terminal_cols = [
    'vehicles_left_at_terminal_outbound',
    'vehicles_left_at_terminal_inbound'
]

# Calculate the total vehicles left at terminal
total_left_at_terminal = jupiter_summer_data[left_at_terminal_cols].fillna(0).sum().sum()

# Calculate the total number of trips made by Jupiter during the summer
total_trips_summer_jupiter = len(jupiter_summer_data)

# Calculate the average amount of vehicles left per trip for Jupiter during summer
avg_left_at_terminal = total_left_at_terminal / total_trips_summer_jupiter if total_trips_summer_jupiter > 0 else 0

# Display trips by month
monthly_trips = jupiter_summer_data.groupby(jupiter_summer_data['time_departure'].dt.month).size()
for month, count in monthly_trips.items():
    month_name = {6: 'June', 7: 'July', 8: 'August'}[month]
    print(f"Month {month} ({month_name}): {count} trips")

print(f"\nTotal vehicles left at terminal (unable to board) for ferry Jupiter during months 6, 7, 8: {total_left_at_terminal}")
print(f"Total number of trips for ferry Jupiter during these months: {total_trips_summer_jupiter}")
print(f"The average number of vehicles left at the terminal per trip: {avg_left_at_terminal:.2f}")


Month 6 (June): 1512 trips
Month 7 (July): 1556 trips
Month 8 (August): 1543 trips

Total vehicles left at terminal (unable to board) for ferry Jupiter during months 6, 7, 8: 14594
Total number of trips for ferry Jupiter during these months: 4611
The average number of vehicles left at the terminal per trip: 3.17


## Q5 Which route has the highest average distance travelled, considering both outbound and inbound directions?

In [30]:
def distance_travelled_analysis(df):
    # Create a copy of the data to avoid SettingWithCopyWarning
    distance_data = df[['route_name', 'ferry_name', 'distance_outbound_nm', 'distance_inbound_nm']].copy()
    
    # Drop null values
    distance_data = distance_data.dropna()
    
    # Calculate the average distance for each leg (outbound and inbound)
    distance_by_route_ferry = distance_data.groupby(['route_name', 'ferry_name'])[['distance_outbound_nm', 'distance_inbound_nm']].mean().reset_index()
    
    # Calculate the overall average distance for each route and ferry
    distance_by_route_ferry['average_distance_nm'] = (distance_by_route_ferry['distance_outbound_nm'] + distance_by_route_ferry['distance_inbound_nm']) / 2
    
    # Group by route_name and calculate the mean of the overall average distance
    average_distance_by_route = distance_by_route_ferry.groupby('route_name')['average_distance_nm'].mean().reset_index()
    
    # Find the route with the highest average distance
    route_with_highest_distance = average_distance_by_route.loc[average_distance_by_route['average_distance_nm'].idxmax()]
    
    # Show context of average distances for all routes
    print("Average distances by route (in nautical miles):")
    for _, row in average_distance_by_route.sort_values('average_distance_nm', ascending=False).iterrows():
        print(f"  {row['route_name']}: {row['average_distance_nm']:.2f} NM")
    
    # Print the final answer
    print(f"\nThe route with the highest average distance travelled is: {route_with_highest_distance['route_name']} with an average distance of {route_with_highest_distance['average_distance_nm']:.2f} NM.")

# Call the function
distance_travelled_analysis(ferry_data)

Average distances by route (in nautical miles):
  Aspöleden: 3.22 NM
  Ljusteröleden: 0.57 NM
  Vaxholmsleden: 0.49 NM
  Oxdjupsleden: 0.25 NM
  Furusundsleden: 0.23 NM

The route with the highest average distance travelled is: Aspöleden with an average distance of 3.22 NM.


## Q6 What is the average distance travelled on Ljusteröleden?

In [31]:
def average_distance_on_ljusteroleden(df):
    ljusteroleden_data = df[df['route_name'] == 'Ljusteröleden'].copy()

    if not ljusteroleden_data.empty:
        # Calculate the average distance for each trip on Ljusteröleden
        ljusteroleden_data['average_trip_distance_nm'] = (
            ljusteroleden_data['distance_outbound_nm'] + ljusteroleden_data['distance_inbound_nm']
        ) / 2

        # Calculate the overall average distance for Ljusteröleden
        average_distance = ljusteroleden_data['average_trip_distance_nm'].mean()

        print(f"The average distance travelled on Ljusteröleden is: {average_distance:.2f} NM.")
    else:
        print("Data for Ljusteröleden not found in the provided DataFrame.")


In [32]:
average_distance_on_ljusteroleden(ferry_data)

The average distance travelled on Ljusteröleden is: 0.57 NM.


## Q7 What is the total Passenger Car Equivalent (PCE) during the peak hour for route Aspöleden in February, combining both outbound and inbound directions?

In [33]:
# Q6 What is the peak hour for route Aspöleden in February?

def peak_hour_aspo_february(df):
    # Make a copy of the dataframe
    df = df.copy()
    
    # Convert time_departure to datetime and filter for February
    df['time_departure'] = pd.to_datetime(df['time_departure'], errors='coerce')
    # Create an explicit copy when filtering to avoid SettingWithCopyWarning
    february_data = df[df['time_departure'].dt.month == 2].copy()
    
    # Extract hour and filter for Aspöleden
    february_data['hour'] = february_data['time_departure'].dt.hour
    aspo_february = february_data[february_data['route_name'] == 'Aspöleden'].copy()
    
    # Aggregate by hour
    hourly_demand = aspo_february.groupby('hour').agg({
        'passenger_car_equivalent_outbound': 'sum',
        'passenger_car_equivalent_inbound': 'sum'
    }).reset_index()
    
    # Find peak hour (for both directions)
    outbound_peak = hourly_demand.loc[hourly_demand['passenger_car_equivalent_outbound'].idxmax()]
    inbound_peak = hourly_demand.loc[hourly_demand['passenger_car_equivalent_inbound'].idxmax()]
    
    hourly_demand['total_pce'] = hourly_demand['passenger_car_equivalent_outbound'] + hourly_demand['passenger_car_equivalent_inbound']
    overall_peak = hourly_demand.loc[hourly_demand['total_pce'].idxmax()]
    
    print(f"Peak hour analysis for Aspöleden in February:")
    print(f"Outbound peak: Hour {outbound_peak['hour']}:00 with {outbound_peak['passenger_car_equivalent_outbound']:.2f} PCE")
    print(f"Inbound peak: Hour {inbound_peak['hour']}:00 with {inbound_peak['passenger_car_equivalent_inbound']:.2f} PCE")
    print(f"The average PCE for the peak hour is: {overall_peak['total_pce']:.2f}")
    
    # Determine overall peak (combining both directions)
   
    
    print(f"\nThe peak hour has a total PCE of {overall_peak['total_pce']:.2f}")
    
    return overall_peak['hour']

# Find the peak hour for Aspöleden in February
peak_hour = peak_hour_aspo_february(ferry_data)

Peak hour analysis for Aspöleden in February:
Outbound peak: Hour 12.0:00 with 701.00 PCE
Inbound peak: Hour 12.0:00 with 791.00 PCE
The average PCE for the peak hour is: 1492.00

The peak hour has a total PCE of 1492.00


## Q8 What is the average fuel consumption on the route Ljusterö to Östanå, combining both outbound and inbound?

In [34]:
# Q7 What is the average fuel consumption on the route Ljusterö to Östanå, combining both outbound and inbound?

def fuel_consumption_ljustero_ostana(df):
    """Calculate average fuel consumption for the route between Ljusterö and Östanå."""
    # Filter for Ljusteröleden route
    ljusteroleden_data = df[df['route_name'] == 'Ljusteröleden'].copy()

    if not ljusteroleden_data.empty:
        # Select fuel consumption columns and drop rows with missing values
        consumption_data = ljusteroleden_data[['fuelcons_outbound_l', 'fuelcons_inbound_l']].dropna()
        
        if not consumption_data.empty:
            # Calculate total fuel consumption per trip (outbound + inbound)
            consumption_data['total_fuel_per_trip'] = consumption_data['fuelcons_outbound_l'] + consumption_data['fuelcons_inbound_l']
            
            # Calculate the average total fuel consumption per trip
            average_consumption = consumption_data['total_fuel_per_trip'].mean()
            
       
            
            # Also provide the breakdown by direction for context
            average_outbound = consumption_data['fuelcons_outbound_l'].mean()
            average_inbound = consumption_data['fuelcons_inbound_l'].mean()
            print(f"  - Average outbound (Ljusterö to Östanå): {average_outbound:.2f} liters")
            print(f"  - Average inbound (Östanå to Ljusterö): {average_inbound:.2f} liters")
            
            print(f"\nThe average combined fuel consumption on the route Ljusterö to Östanå: {average_consumption:.2f} liters per trip")
            
            return average_consumption
        else:
            print("No valid fuel consumption data found for the route between Ljusterö and Östanå")
            return None
    else:
        print("Data for Ljusteröleden not found in the provided DataFrame")
        return None

# Calculate average fuel consumption for the route between Ljusterö and Östanå
average_fuel_consumption = fuel_consumption_ljustero_ostana(ferry_data)

  - Average outbound (Ljusterö to Östanå): 7.00 liters
  - Average inbound (Östanå to Ljusterö): 6.98 liters

The average combined fuel consumption on the route Ljusterö to Östanå: 13.98 liters per trip


# Q9: What is the average trip duration for Oxdjupsleden?

In [35]:
# Q9: What is the average trip duration for Oxdjupsleden?

def calculate_oxdjupsleden_duration(df):
    """Calculate the average trip duration for Oxdjupsleden route."""
    # Create a copy to avoid modifying original data
    oxdjup_data = df[df['route_name'] == 'Oxdjupsleden'].copy()
    
    # Convert time columns to datetime
    time_columns = ['start_time_outbound', 'end_time_outbound', 
                    'start_time_inbound', 'end_time_inbound']
    for col in time_columns:
        oxdjup_data[col] = pd.to_datetime(oxdjup_data[col])
    
    # Calculate duration in minutes for outbound and inbound trips
    oxdjup_data.loc[:, 'outbound_duration'] = (
        (oxdjup_data['end_time_outbound'] - oxdjup_data['start_time_outbound'])
        .dt.total_seconds() / 60
    )
    
    oxdjup_data.loc[:, 'inbound_duration'] = (
        (oxdjup_data['end_time_inbound'] - oxdjup_data['start_time_inbound'])
        .dt.total_seconds() / 60
    )
    
    # Calculate average durations
    avg_outbound = oxdjup_data['outbound_duration'].mean()
    avg_inbound = oxdjup_data['inbound_duration'].mean()
    avg_total = (avg_outbound + avg_inbound) / 2
    
    print(f"Average Trip Duration for Oxdjupsleden:")
    print(f"  - Outbound: {avg_outbound:.2f} minutes")
    print(f"  - Inbound: {avg_inbound:.2f} minutes")
    print(f"\nThe average trip duration is {avg_total:.2f} minutes")
    
    return avg_total

# Calculate average trip duration for Oxdjupsleden
avg_duration = calculate_oxdjupsleden_duration(ferry_data)

Average Trip Duration for Oxdjupsleden:
  - Outbound: 3.16 minutes
  - Inbound: 3.09 minutes

The average trip duration is 3.12 minutes


# Q10: Which ferry has the best fuel efficiency, and what is its liters per nautical mile?

In [36]:
# Q10: Which ferry has the best fuel efficiency, and what is its liters per nautical mile?

def analyze_fuel_efficiency(df):
    """
    Calculate fuel efficiency (liters per nautical mile) for each ferry.
    """
    # Create a copy to avoid modifying original data
    efficiency_data = df.copy()
    
    # Select relevant columns and drop rows with missing values
    efficiency_data = efficiency_data[[
        'ferry_name',
        'fuelcons_outbound_l',
        'fuelcons_inbound_l',
        'distance_outbound_nm',
        'distance_inbound_nm'
    ]].dropna()
    
    if efficiency_data.empty:
        print("No valid data found for fuel efficiency analysis")
        return None
    
    # Calculate total fuel consumption and distance for each ferry
    ferry_efficiency = efficiency_data.groupby('ferry_name').agg({
        'fuelcons_outbound_l': 'sum',
        'fuelcons_inbound_l': 'sum',
        'distance_outbound_nm': 'sum',
        'distance_inbound_nm': 'sum'
    })
    
    # Calculate total fuel consumption and distance
    ferry_efficiency['total_fuel_l'] = (ferry_efficiency['fuelcons_outbound_l'] + 
                                       ferry_efficiency['fuelcons_inbound_l'])
    ferry_efficiency['total_distance_nm'] = (ferry_efficiency['distance_outbound_nm'] + 
                                           ferry_efficiency['distance_inbound_nm'])
    
    # Calculate fuel efficiency (liters per nautical mile)
    ferry_efficiency['efficiency_l_per_nm'] = (ferry_efficiency['total_fuel_l'] / 
                                             ferry_efficiency['total_distance_nm'])
    
    # Sort by efficiency (ascending order - lower is better)
    ferry_efficiency = ferry_efficiency.sort_values('efficiency_l_per_nm')
    
    # Print results
    print("\nFuel Efficiency Analysis (liters per nautical mile):")
    print("Lower values indicate better fuel efficiency")
    print("-" * 50)
    for ferry, row in ferry_efficiency.iterrows():
        print(f"{ferry}: {row['efficiency_l_per_nm']:.2f} L/NM")
    
    # Print the most efficient ferry
    most_efficient = ferry_efficiency.index[0]
    print(f"\nThe most fuel-efficient ferry is: {most_efficient}: {ferry_efficiency['efficiency_l_per_nm'].iloc[0]:.2f} L/NM")
    
    return ferry_efficiency

# Call the function
ferry_efficiency = analyze_fuel_efficiency(ferry_data)


Fuel Efficiency Analysis (liters per nautical mile):
Lower values indicate better fuel efficiency
--------------------------------------------------
Merkurius: 6.08 L/NM
Nina: 8.37 L/NM
Yxlan: 10.82 L/NM
Fragancia: 11.92 L/NM
Jupiter: 12.30 L/NM

The most fuel-efficient ferry is: Merkurius: 6.08 L/NM


# Q11: What is the most underutilized route in the dataset?

In [37]:
def analyze_route_utilization(df):
    """
    Analyze route utilization by comparing actual vehicle load to capacity.
    This helps identify which routes are most underutilized in terms of actual vehicle load.
    """
    # Create a copy to avoid modifying original data
    utilization_data = df.copy()
    
    # Calculate total vehicles per trip (both directions)
    utilization_data['total_vehicles_outbound'] = (
        utilization_data['cars_outbound'] + 
        utilization_data['trucks_outbound'] + 
        utilization_data['trucks_with_trailer_outbound'] +
        utilization_data['motorcycles_outbound'] +
        utilization_data['buses_outbound']
    )
    
    utilization_data['total_vehicles_inbound'] = (
        utilization_data['cars_inbound'] + 
        utilization_data['trucks_inbound'] + 
        utilization_data['trucks_with_trailer_inbound'] +
        utilization_data['motorcycles_inbound'] +
        utilization_data['buses_inbound']
    )
    
    # Group by route and calculate average vehicle load
    route_stats = utilization_data.groupby('route_name').agg({
        'total_vehicles_outbound': 'mean',
        'total_vehicles_inbound': 'mean',
        'ferry_name': 'count'  # Number of trips
    }).rename(columns={'ferry_name': 'trip_count'})
    
    # Calculate average vehicles per trip (both directions)
    route_stats['avg_vehicles_per_trip'] = (
        (route_stats['total_vehicles_outbound'] + route_stats['total_vehicles_inbound']) / 2
    ).round(2)
    
    # Find the route with highest average vehicles (assumed to be closest to capacity)
    max_vehicles = route_stats['avg_vehicles_per_trip'].max()
    
    # Calculate utilization percentage relative to the most utilized route
    route_stats['utilization_percentage'] = (
        (route_stats['avg_vehicles_per_trip'] / max_vehicles) * 100
    ).round(2)
    
    # Sort by utilization percentage (ascending to show most underutilized first)
    route_stats = route_stats.sort_values('utilization_percentage')
    
    # Print results
    print("\nRoute Utilization Analysis:")
    print("Lower percentages indicate more underutilized routes")
    print("-" * 60)
    
    for route, row in route_stats.iterrows():
        print(f"\n{route}:")
        print(f"  - Average vehicles per trip: {row['avg_vehicles_per_trip']}")
        print(f"  - Utilization percentage: {row['utilization_percentage']:.2f}%")
        print(f"  - Number of trips analyzed: {row['trip_count']}")
    
    # Identify the most underutilized route
    most_underutilized = route_stats.index[0]
    print(f"\nMost underutilized route: {most_underutilized}")
    print(f"Average vehicles per trip: {route_stats['avg_vehicles_per_trip'].iloc[0]}")
    print(f"Utilization percentage: {route_stats['utilization_percentage'].iloc[0]:.2f}%")
    
    return route_stats

# Call the function
route_utilization = analyze_route_utilization(ferry_data)
route_utilization


Route Utilization Analysis:
Lower percentages indicate more underutilized routes
------------------------------------------------------------

Oxdjupsleden:
  - Average vehicles per trip: 9.31
  - Utilization percentage: 49.68%
  - Number of trips analyzed: 17505.0

Aspöleden:
  - Average vehicles per trip: 13.0
  - Utilization percentage: 69.37%
  - Number of trips analyzed: 5514.0

Furusundsleden:
  - Average vehicles per trip: 14.41
  - Utilization percentage: 76.89%
  - Number of trips analyzed: 15542.0

Vaxholmsleden:
  - Average vehicles per trip: 16.24
  - Utilization percentage: 86.66%
  - Number of trips analyzed: 17517.0

Ljusteröleden:
  - Average vehicles per trip: 18.74
  - Utilization percentage: 100.00%
  - Number of trips analyzed: 13280.0

Most underutilized route: Oxdjupsleden
Average vehicles per trip: 9.31
Utilization percentage: 49.68%


,total_vehicles_outbound,total_vehicles_inbound,trip_count,avg_vehicles_per_trip,utilization_percentage
route_name,,,,,
Oxdjupsleden,9.349272,9.261697,17505,9.31,49.68
Aspöleden,13.170294,12.830432,5514,13.00,69.37
Furusundsleden,14.239094,14.590143,15542,14.41,76.89
Vaxholmsleden,15.971171,16.500999,17517,16.24,86.66
Ljusteröleden,19.145407,18.342395,13280,18.74,100.00


# Q12: What's the average number of vehicles left at terminals in March for Furusundsleden? 

In [38]:
# Q12: What's the average number of vehicles left at terminals in June for Furusundsleden?

def average_vehicles_left_per_month(df):
    """
    Calculates the average number of vehicles left at terminals per month,
    with specific focus on Furusundsleden in June.
    """
    # Create a copy to avoid warnings
    df = df.copy()
    
    # 1. Calculate vehicles left at terminal (using actual columns for this)
    df['total_vehicles_left'] = df['vehicles_left_at_terminal_outbound'].fillna(0) + df['vehicles_left_at_terminal_inbound'].fillna(0)

    # 2. Extract Month from 'time_departure'
    df['time_departure'] = pd.to_datetime(df['time_departure'], errors='coerce')
    df['month'] = df['time_departure'].dt.strftime('%Y-%m')

    # 3. Group by both terminal and route, calculate averages
    # First by terminal and month
    terminal_avg = df.groupby(['terminal_departure', 'month'])['total_vehicles_left'].mean().reset_index()
    terminal_avg = terminal_avg.rename(columns={'terminal_departure': 'terminal', 
                                               'total_vehicles_left': 'avg_vehicles_left'})
    
    # Then by route and month
    route_avg = df.groupby(['route_name', 'month'])['total_vehicles_left'].mean().reset_index()
    route_avg = route_avg.rename(columns={'total_vehicles_left': 'avg_vehicles_left'})
    
    # 4. Filter for June data
    june_terminal_data = terminal_avg[terminal_avg['month'].str.contains('-06')]
    june_route_data = route_avg[route_avg['month'].str.contains('-06')]
    
    # 5. Get specific answer for Furusundsleden in June
    furusund_june = june_route_data[june_route_data['route_name'] == 'Furusundsleden']
    
    # Display results
    print("Average Vehicles Left at Terminals Per Month:")
    print(terminal_avg)
    
    print("\nAverage Vehicles Left at Terminals in June by Route:")
    print(june_route_data)
    
    # Provide specific answer for Furusundsleden in June
    if not furusund_june.empty:
        furusund_avg = furusund_june['avg_vehicles_left'].values[0]
        print(f"\nThe average number of vehicles left at terminals in June for Furusundsleden is {furusund_avg:.2f}")
    else:
        print("\nNo data available for Furusundsleden in June")
        
    return route_avg

# Calculate and display results
route_data = average_vehicles_left_per_month(ferry_data)

Average Vehicles Left at Terminals Per Month:
                                             terminal    month  \
0   Furusund                                      ...  2023-03   
1   Furusund                                      ...  2023-04   
2   Furusund                                      ...  2023-05   
3   Furusund                                      ...  2023-06   
4   Furusund                                      ...  2023-07   
5   Furusund                                      ...  2023-08   
6   Furusund                                      ...  2023-09   
7   Furusund                                      ...  2023-10   
8   Furusund                                      ...  2023-11   
9   Furusund                                      ...  2023-12   
10  Furusund                                      ...  2024-01   
11  Furusund                                      ...  2024-02   
12  Karlskrona                                    ...  2023-03   
13  Karlskrona                

# Q13: Which route has the highest vehicle throughput per day?

In [39]:
import pandas as pd

def highest_vehicle_throughput_per_day(df):
    """
    Determines the route with the highest vehicle throughput per day.

    Args:
        df: pandas DataFrame containing ferry operation data.
            It should have columns 'route_name' (string), 'time_departure' (string),
            'cars_outbound' (numeric), 'trucks_outbound' (numeric),
            and 'trucks_with_trailer_outbound' (numeric).

    Returns:
        pandas DataFrame: A DataFrame with the route name and its corresponding
                          highest daily vehicle throughput,
                          or an empty DataFrame if there are no data.
    """

    # 1. Calculate Total Vehicles
    df['total_vehicles'] = df['cars_outbound'].fillna(0) + df['trucks_outbound'].fillna(0) + df['trucks_with_trailer_outbound'].fillna(0)

    # 2. Extract Date from 'time_departure'
    if 'time_departure' not in df.columns:
        print("Error: 'time_departure' column not found in DataFrame.")
        return pd.DataFrame()

    df['date'] = pd.to_datetime(df['time_departure']).dt.date

    # 3. Calculate Daily Vehicle Throughput for Each Route
    daily_throughput = df.groupby(['route_name', 'date'])['total_vehicles'].sum().reset_index()

    # 4. Find the Highest Daily Throughput for Each Route
    highest_daily_throughput = daily_throughput.groupby('route_name')['total_vehicles'].max().reset_index()

    # 5. Rename the 'total_vehicles' column for clarity
    highest_daily_throughput = highest_daily_throughput.rename(columns={'total_vehicles': 'highest_daily_throughput'})

    # 6. Sort the results in descending order of throughput
    highest_daily_throughput = highest_daily_throughput.sort_values('highest_daily_throughput', ascending=False)

    # 7. Find the route with the overall maximum throughput
    top_route = highest_daily_throughput.iloc[0]
    
    print("Highest Vehicle Throughput per Day by Route:")
    for i, (_, row) in enumerate(highest_daily_throughput.iterrows(), 1):
        print(f"{i}. {row['route_name']}: {int(row['highest_daily_throughput'])} vehicles")
    
    # Print a clear answer for Q13
    print(f"\nQ13 Answer: The route with the highest vehicle throughput per day is {top_route['route_name']} with {int(top_route['highest_daily_throughput'])} vehicles.")

    return highest_daily_throughput

# Load the ferry trips data
try:
    ferry_data = pd.read_csv('data/ferry_trips_data_large.csv')
    
    # Execute the function
    highest_vehicle_throughput_per_day(ferry_data)
    
except Exception as e:
    print(f"Error: {e}")

Highest Vehicle Throughput per Day by Route:
1. Ljusteröleden: 1840 vehicles
2. Furusundsleden: 1794 vehicles
3. Vaxholmsleden: 1387 vehicles
4. Oxdjupsleden: 841 vehicles
5. Aspöleden: 351 vehicles

Q13 Answer: The route with the highest vehicle throughput per day is Ljusteröleden with 1840 vehicles.


Q: What is Maximum Passenger Car Equivalent (PCE) during the peak hour of Aspöleden route in February?


# Q14: What's the busiest day of the week for route Oxdjupsleden?

In [40]:
# Q14: What's the busiest day of the week for route Oxdjupsleden?

def busiest_day_of_week_oxdjupsleden(df):
    """
    Determines the busiest day of the week for the Oxdjupsleden route.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        pandas DataFrame: Statistics about traffic by day of week for Oxdjupsleden
    """
    # Filter for Oxdjupsleden route
    oxdjup_data = df[df['route_name'] == 'Oxdjupsleden'].copy()
    
    if oxdjup_data.empty:
        print("No data available for Oxdjupsleden route")
        return pd.DataFrame()
    
    # Convert departure time to datetime if it's not already
    oxdjup_data['time_departure'] = pd.to_datetime(oxdjup_data['time_departure'])
    
    # Extract day of week (0=Monday, 6=Sunday)
    oxdjup_data['day_of_week'] = oxdjup_data['time_departure'].dt.dayofweek
    
    # Create a mapping for day names
    day_names = {
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    }
    
    # Apply mapping to get day names
    oxdjup_data['day_name'] = oxdjup_data['day_of_week'].map(day_names)
    
    # Calculate total passenger car equivalent for each trip (both directions)
    if 'passenger_car_equivalent_outbound_and_inbound' in oxdjup_data.columns:
        oxdjup_data['total_pce'] = oxdjup_data['passenger_car_equivalent_outbound_and_inbound']
    else:
        # If combined PCE isn't available, calculate from both directions
        oxdjup_data['total_pce'] = oxdjup_data['passenger_car_equivalent_outbound'].fillna(0) + \
                                  oxdjup_data['passenger_car_equivalent_inbound'].fillna(0)
    
    # Group by day of week and aggregate various metrics (removing total_vehicles_left)
    daily_stats = oxdjup_data.groupby(['day_of_week', 'day_name']).agg(
        total_trips=('route_name', 'count'),
        avg_pce_per_trip=('total_pce', 'mean'),
        total_pce=('total_pce', 'sum')
    ).reset_index()
    
    # Sort by day of week for easier reading
    daily_stats = daily_stats.sort_values('day_of_week')
    
    # Find the busiest day based on total PCE (passenger car equivalent)
    busiest_day_row = daily_stats.loc[daily_stats['total_pce'].idxmax()]
    
    # Display the results
    print("Traffic statistics by day of week for Oxdjupsleden:")
    print(daily_stats)
    
    print(f"\nAnswer to Q14: The busiest day of the week for route Oxdjupsleden is {busiest_day_row['day_name']}")
    print(f"On {busiest_day_row['day_name']}s, Oxdjupsleden has:")
    print(f"- {busiest_day_row['total_trips']:.0f} trips in total")
    print(f"- {busiest_day_row['avg_pce_per_trip']:.2f} passenger car equivalents per trip on average")
    print(f"- {busiest_day_row['total_pce']:.2f} total passenger car equivalents")
    
    return daily_stats

# Execute the function
oxdjup_daily_traffic = busiest_day_of_week_oxdjupsleden(ferry_data)

Traffic statistics by day of week for Oxdjupsleden:
   day_of_week   day_name  total_trips  avg_pce_per_trip  total_pce
0            0     Monday         2651         18.468502    48960.0
1            1    Tuesday         2696         19.153932    51639.0
2            2  Wednesday         2771         19.490076    54007.0
3            3   Thursday         2771         20.075424    55629.0
4            4     Friday         2744         19.108054    52432.5
5            5   Saturday         2010         22.208458    44639.0
6            6     Sunday         1862         21.922395    40819.5

Answer to Q14: The busiest day of the week for route Oxdjupsleden is Thursday
On Thursdays, Oxdjupsleden has:
- 2771 trips in total
- 20.08 passenger car equivalents per trip on average
- 55629.00 total passenger car equivalents



# Q15: Which hour of the day has the highest average passenger car equivalent (PCE) for route Ljusteröleden (with inbound and outbound trips combined)?

In [41]:
def analyze_pce_variation_ljusteroleden(df):
    """
    Analyzes how the passenger car equivalent (PCE) varies on average throughout the day
    for route Ljusteröleden.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        DataFrame: Hourly statistics of PCE for Ljusteröleden route
    """
    # Filter for Ljusteröleden route
    ljustero_data = df[df['route_name'] == 'Ljusteröleden'].copy()
    
    if ljustero_data.empty:
        print("No data available for Ljusteröleden route")
        return pd.DataFrame()
    
    # Convert departure time to datetime
    ljustero_data['time_departure'] = pd.to_datetime(ljustero_data['time_departure'])
    
    # Extract hour of day
    ljustero_data['hour_of_day'] = ljustero_data['time_departure'].dt.hour
    
    # Calculate total PCE for each trip (outbound + inbound)
    ljustero_data['total_pce'] = (
        ljustero_data['passenger_car_equivalent_outbound'].fillna(0) + 
        ljustero_data['passenger_car_equivalent_inbound'].fillna(0)
    )
    
    # Group by hour of day and calculate statistics
    hourly_stats = ljustero_data.groupby('hour_of_day').agg(
        avg_pce_outbound=('passenger_car_equivalent_outbound', 'mean'),
        avg_pce_inbound=('passenger_car_equivalent_inbound', 'mean'),
        avg_total_pce=('total_pce', 'mean'),
        trip_count=('route_id', 'count')
    ).reset_index()
    
    # Ensure we have all hours (0-23)
    full_hours = pd.DataFrame({'hour_of_day': range(0, 24)})
    hourly_stats = pd.merge(full_hours, hourly_stats, on='hour_of_day', how='left').fillna(0)
    
    # Sort by hour of day
    hourly_stats = hourly_stats.sort_values('hour_of_day')
    
    # Find the peak hours and PCE values
    peak_hour_outbound = hourly_stats.loc[hourly_stats['avg_pce_outbound'].idxmax()]
    peak_hour_inbound = hourly_stats.loc[hourly_stats['avg_pce_inbound'].idxmax()]
    
    # Find the hour with the highest combined PCE (outbound + inbound)
    max_combined_hour = hourly_stats.loc[hourly_stats['avg_total_pce'].idxmax()]
    
    print("\nPeak PCE times for Ljusteröleden:")
    print(f"Outbound: Hour {peak_hour_outbound['hour_of_day']} with average PCE of {peak_hour_outbound['avg_pce_outbound']:.2f}")
    print(f"Inbound: Hour {peak_hour_inbound['hour_of_day']} with average PCE of {peak_hour_inbound['avg_pce_inbound']:.2f}")
    print(f"Combined average: {max_combined_hour['avg_total_pce']:.2f}")
    
    print(f"\nThe hour with the highest combined average PCE is no.: {max_combined_hour['hour_of_day']}")
    
    return hourly_stats

# Execute the function with the already loaded ferry_data
hourly_pce_stats = analyze_pce_variation_ljusteroleden(ferry_data)


Peak PCE times for Ljusteröleden:
Outbound: Hour 15.0 with average PCE of 35.49
Inbound: Hour 15.0 with average PCE of 36.63
Combined average: 72.12

The hour with the highest combined average PCE is no.: 15.0



# Q16 What is the average passenger car equivalent (PCE) during June, July, August for Vaxholmsleden route?


In [42]:
def calculate_summer_pce_vaxholmsleden(df):
    """
    Calculates the average passenger car equivalent (PCE) during summer months 
    for the Vaxholmsleden route.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        float: Average PCE during summer for Vaxholmsleden
    """
    # Make a copy of the dataframe to avoid warnings
    df_copy = df.copy()
    
    # Convert time_departure to datetime for filtering
    df_copy['time_departure'] = pd.to_datetime(df_copy['time_departure'], errors='coerce')
    
    # Filter data for Vaxholmsleden route
    vaxholm_data = df_copy[df_copy['route_name'] == 'Vaxholmsleden']
    
    if vaxholm_data.empty:
        print("No data found for Vaxholmsleden route")
        return None
    
    # Define summer months (June, July, August)
    summer_months = [6, 7, 8]
    
    # Filter for summer months
    summer_data = vaxholm_data[vaxholm_data['time_departure'].dt.month.isin(summer_months)]
    
    if summer_data.empty:
        print("No summer data found for Vaxholmsleden route")
        return None
    
    # Calculate average PCE by adding outbound and inbound PCEs directly
    outbound_pce = summer_data['passenger_car_equivalent_outbound'].fillna(0)
    inbound_pce = summer_data['passenger_car_equivalent_inbound'].fillna(0)
    total_pce = outbound_pce + inbound_pce
    
    # Calculate average PCE
    avg_summer_pce = total_pce.mean()
    
    # Calculate average PCE by month for context
    monthly_pce = summer_data.groupby(summer_data['time_departure'].dt.month).agg({
        'passenger_car_equivalent_outbound': lambda x: x.fillna(0).mean(),
        'passenger_car_equivalent_inbound': lambda x: x.fillna(0).mean()
    })
    
    monthly_pce['total_avg_pce'] = monthly_pce['passenger_car_equivalent_outbound'] + monthly_pce['passenger_car_equivalent_inbound']
    
    # Map month numbers to names for better readability
    month_names = {6: 'June', 7: 'July', 8: 'August'}
    monthly_pce.index = monthly_pce.index.map(month_names)
    
    print("Relevant context:")
    print(f"Monthly breakdown of average PCE for Vaxholmsleden during summer:")
    for month, row in monthly_pce.iterrows():
        print(f"{month}: {row['total_avg_pce']:.2f} PCE")
    print(f'Combined average PCE: {avg_summer_pce:.2f}.')
    
    print(f"\nQ21 Answer: The average PCE during June, July and August for Vaxholmsleden route is {avg_summer_pce:.2f}.")
    
    return avg_summer_pce

# Execute the function
avg_summer_pce = calculate_summer_pce_vaxholmsleden(ferry_data)

Relevant context:
Monthly breakdown of average PCE for Vaxholmsleden during summer:
June: 46.41 PCE
July: 39.54 PCE
August: 40.53 PCE
Combined average PCE: 42.06.

Q21 Answer: The average PCE during June, July and August for Vaxholmsleden route is 42.06.


# Q17: Which route has the highest average capacity utilization rate?

In [43]:
import pandas as pd

def calculate_route_capacity_utilization(df):
    # Create a copy to avoid modifying the original dataframe
    utilization_data = df.copy()
    
    # Assume that passenger_car_equivalent represents the actual load
    # Maximum capacity values for each route
    route_max_capacity = {
        'Aspöleden': 60,
        'Furusundsleden': 60,
        'Ljusteröleden': 75,
        'Oxdjupsleden': 50,
        'Vaxholmsleden': 65
    }
    
    # Function to calculate utilization for each trip
    def calculate_utilization(row):
        route = row['route_name']
        if route in route_max_capacity:
            outbound_pce = row['passenger_car_equivalent_outbound'] if not pd.isna(row['passenger_car_equivalent_outbound']) else 0
            inbound_pce = row['passenger_car_equivalent_inbound'] if not pd.isna(row['passenger_car_equivalent_inbound']) else 0
            max_cap = route_max_capacity[route]
            
            # Calculate utilization for each direction
            outbound_util = min(outbound_pce / max_cap * 100, 100) if max_cap > 0 else 0
            inbound_util = min(inbound_pce / max_cap * 100, 100) if max_cap > 0 else 0
            
            # Average of both directions
            return (outbound_util + inbound_util) / 2
        return None
    
    # Apply the utilization calculation
    utilization_data['capacity_utilization'] = utilization_data.apply(calculate_utilization, axis=1)
    
    # Group by route and calculate average utilization
    route_utilization = utilization_data.groupby('route_name')['capacity_utilization'].mean().reset_index()
    
    # Sort by utilization rate (descending)
    route_utilization = route_utilization.sort_values('capacity_utilization', ascending=False)
    
    # Print ONLY the requested outputs
    print("Average Capacity Utilization Rate per Route:")
    for _, row in route_utilization.iterrows():
        print(f"{row['route_name']}: {row['capacity_utilization']:.2f}%")
    
    # Get the highest utilization route
    highest_util_route = route_utilization.iloc[0]
    
    # Print ONLY the direct answer
    print(f"\nThe route with the highest average capacity utilization rate is {highest_util_route['route_name']} at {highest_util_route['capacity_utilization']:.2f}%.")

# Load the ferry trips data
try:

    # Execute the function
    calculate_route_capacity_utilization(ferry_data)
    
except Exception as e:
    print(f"Error: {e}")

Average Capacity Utilization Rate per Route:
Ljusteröleden: 30.22%
Furusundsleden: 29.92%
Vaxholmsleden: 28.28%
Aspöleden: 24.06%
Oxdjupsleden: 19.88%

The route with the highest average capacity utilization rate is Ljusteröleden at 30.22%.


# Q18: At which hour do routes most frequently reach peak capacity?

In [44]:
import pandas as pd

def analyze_peak_capacity_frequency(df):
    """
    Analyzes when ferry routes reach peak capacity most frequently.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        DataFrame: containing data on peak capacity frequency by hour and day
    """
    # Create a copy to avoid modifying the original dataframe
    peak_data = df.copy()
    
    # Convert time_departure to datetime and extract time components
    peak_data['time_departure'] = pd.to_datetime(peak_data['time_departure'])
    peak_data['hour_of_day'] = peak_data['time_departure'].dt.hour
    peak_data['day_of_week'] = peak_data['time_departure'].dt.dayofweek
    peak_data['day_name'] = peak_data['time_departure'].dt.day_name()
    
    # Define peak capacity thresholds for each route (e.g., 80% of max capacity)
    route_peak_threshold = {
        'Aspöleden': 48,  # 80% of 60
        'Furusundsleden': 48,  # 80% of 60
        'Ljusteröleden': 60,  # 80% of 75
        'Oxdjupsleden': 40,  # 80% of 50
        'Vaxholmsleden': 52   # 80% of 65
    }
    
    # Function to check if a trip is at peak capacity
    def is_peak_capacity(row):
        route = row['route_name']
        if route in route_peak_threshold:
            outbound_pce = row['passenger_car_equivalent_outbound'] if not pd.isna(row['passenger_car_equivalent_outbound']) else 0
            inbound_pce = row['passenger_car_equivalent_inbound'] if not pd.isna(row['passenger_car_equivalent_inbound']) else 0
            threshold = route_peak_threshold[route]
            
            # Consider peak if either direction reaches peak capacity
            return 1 if outbound_pce >= threshold or inbound_pce >= threshold else 0
        return 0
    
    # Apply the peak capacity check
    peak_data['is_peak'] = peak_data.apply(is_peak_capacity, axis=1)
    
    # Calculate peak capacity frequency by hour of day
    peak_by_hour = peak_data.groupby(['hour_of_day']).agg(
        total_trips=('route_name', 'count'),
        peak_trips=('is_peak', 'sum')
    ).reset_index()
    
    peak_by_hour['peak_percentage'] = (peak_by_hour['peak_trips'] / peak_by_hour['total_trips'] * 100).round(2)
    peak_by_hour = peak_by_hour.sort_values('peak_percentage', ascending=False)
    
    # Get top peak hour
    top_peak_hour = peak_by_hour.iloc[0]
    
    # Print only the hour results (top 5)
    print("Peak Capacity by Hour of Day (top 5):")
    print("   hour_of_day  total_trips  peak_trips  peak_percentage")
    for i, (_, row) in enumerate(peak_by_hour.head(5).iterrows(), 1):
        print(f"{i}    {int(row['hour_of_day'])}        {row['total_trips']}    {row['peak_trips']}    {row['peak_percentage']}")
    
    # Print the direct answer with emphasis on the hour
    print(f"\nRoutes reach peak capacity most frequently at hour {int(top_peak_hour['hour_of_day'])}.")
    
    return peak_by_hour

# Load the ferry trips data
try:
    # Execute the function
    analyze_peak_capacity_frequency(ferry_data)
    
except Exception as e:
    print(f"Error: {e}") 

Peak Capacity by Hour of Day (top 5):
   hour_of_day  total_trips  peak_trips  peak_percentage
1    15        3838.0    347.0    9.04
2    12        3797.0    281.0    7.4
3    11        3576.0    211.0    5.9
4    14        3961.0    229.0    5.78
5    16        4285.0    243.0    5.67

Routes reach peak capacity most frequently at hour 15.


# Q19: Which ferry route had the highest ratio of pedestrians to vehicles in 2023?

In [45]:
def analyze_pedestrian_to_vehicle_ratio(df):
    """
    Analyzes which ferry route had the highest ratio of pedestrians to vehicles in 2023.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        str: Name of the route with highest pedestrian to vehicle ratio
    """
    # Create a copy to avoid modifying the original dataframe
    analysis_data = df.copy()
    
    # Convert time_departure to datetime for filtering
    analysis_data['time_departure'] = pd.to_datetime(analysis_data['time_departure'])
    
    # Filter data for 2023 and create an explicit copy to avoid the SettingWithCopyWarning
    data_2023 = analysis_data[analysis_data['time_departure'].dt.year == 2023].copy()
    
    # Calculate total pedestrians and vehicles for each trip
    data_2023['total_pedestrians'] = (
        data_2023['pedestrians_outbound'].fillna(0) + 
        data_2023['pedestrians_inbound'].fillna(0)
    )
    
    # Calculate total vehicles (cars, trucks, motorcycles, buses)
    data_2023['total_vehicles'] = (
        data_2023['cars_outbound'].fillna(0) + 
        data_2023['trucks_outbound'].fillna(0) + 
        data_2023['trucks_with_trailer_outbound'].fillna(0) + 
        data_2023['motorcycles_outbound'].fillna(0) + 
        data_2023['buses_outbound'].fillna(0) + 
        data_2023['cars_inbound'].fillna(0) + 
        data_2023['trucks_inbound'].fillna(0) + 
        data_2023['trucks_with_trailer_inbound'].fillna(0) + 
        data_2023['motorcycles_inbound'].fillna(0) + 
        data_2023['buses_inbound'].fillna(0)
    )
    
    # Replace zero vehicles with a small value to avoid division by zero
    data_2023['total_vehicles'] = data_2023['total_vehicles'].replace(0, 0.1)
    
    # Calculate the pedestrian to vehicle ratio using .loc to avoid warning
    data_2023.loc[:, 'ped_to_vehicle_ratio'] = data_2023['total_pedestrians'] / data_2023['total_vehicles']
    
    # Calculate average ratio per route
    route_ratios = data_2023.groupby('route_name').agg(
        avg_ratio=('ped_to_vehicle_ratio', 'mean'),
        total_pedestrians=('total_pedestrians', 'sum'),
        total_vehicles=('total_vehicles', 'sum'),
        trip_count=('route_id', 'count')
    ).reset_index()
    
    # Calculate overall ratio for each route
    route_ratios['overall_ratio'] = route_ratios['total_pedestrians'] / route_ratios['total_vehicles']
    
    # Sort by average ratio in descending order
    route_ratios = route_ratios.sort_values('avg_ratio', ascending=False)
    
    # Get the route with highest ratio
    top_route = route_ratios.iloc[0]
    
    # Print detailed results
    print("Analysis of Pedestrian to Vehicle Ratios in 2023:")
    print("\nAverage Pedestrian-to-Vehicle Ratio by Route:")
    print("  Route Name        Avg Ratio    Overall Ratio    Total Pedestrians    Total Vehicles    Trips")
    for i, (_, row) in enumerate(route_ratios.iterrows(), 1):
        print(f"{i}. {row['route_name']:<20} {row['avg_ratio']:.3f}        {row['overall_ratio']:.3f}            {int(row['total_pedestrians']):,}               {int(row['total_vehicles']):,}         {row['trip_count']:,}")
    
    # Print the answer
    print(f"\nAnswer: {top_route['route_name']} had the highest ratio of pedestrians to vehicles in 2023 with an average ratio of {top_route['avg_ratio']:.3f}.")
    
    return top_route['route_name']

# Execute the function
highest_pedestrian_ratio_route = analyze_pedestrian_to_vehicle_ratio(ferry_data)

Analysis of Pedestrian to Vehicle Ratios in 2023:

Average Pedestrian-to-Vehicle Ratio by Route:
  Route Name        Avg Ratio    Overall Ratio    Total Pedestrians    Total Vehicles    Trips
1. Aspöleden            0.859        0.614            76,522               124,697         4,631
2. Vaxholmsleden        0.235        0.189            93,040               492,584         14,610
3. Furusundsleden       0.029        0.015            6,096               399,140         13,120
4. Oxdjupsleden         0.027        0.028            8,079               292,623         14,607
5. Ljusteröleden        0.008        0.007            2,801               428,881         11,049

Answer: Aspöleden had the highest ratio of pedestrians to vehicles in 2023 with an average ratio of 0.859.


# Q20: What is the total CO2 emission from Ljusteröleden in February 2024, assuming 2.68 kg CO2 per liter of fuel?

In [46]:
def calculate_ljustero_co2_emissions_feb2024(df):
    """
    Calculates the total CO2 emissions from Ljusteröleden ferry route in February 2024,
    assuming 2.68 kg CO2 per liter of fuel.
    
    Args:
        df: pandas DataFrame containing ferry operation data
        
    Returns:
        float: total CO2 emissions in kg
    """
    # Create a copy to avoid modifying the original dataframe
    emissions_data = df.copy()
    
    # Convert time_departure to datetime for filtering by year and month
    emissions_data['time_departure'] = pd.to_datetime(emissions_data['time_departure'])
    
    # Filter data for Ljusteröleden in February 2024
    filtered_data = emissions_data[
        (emissions_data['route_name'] == 'Ljusteröleden') & 
        (emissions_data['time_departure'].dt.year == 2024) & 
        (emissions_data['time_departure'].dt.month == 2)
    ]
    
    # Sum up fuel consumption (both outbound and inbound)
    total_fuel_outbound = filtered_data['fuelcons_outbound_l'].sum()
    total_fuel_inbound = filtered_data['fuelcons_inbound_l'].sum()
    total_fuel_consumption = total_fuel_outbound + total_fuel_inbound
    
    # CO2 emission factor: 2.68 kg CO2 per liter of fuel
    co2_factor = 2.68
    total_co2_emissions = total_fuel_consumption * co2_factor
    
    # Convert to tons for easier readability if emissions are large
    total_co2_emissions_tons = total_co2_emissions / 1000
    
    # Print the results
    print("CO2 Emissions Analysis for Ljusteröleden in February 2024:")
    print(f"Total fuel consumption: {total_fuel_consumption:,.2f} liters")
    print(f"CO2 emission factor: {co2_factor} kg CO2 per liter")
    print(f"Total CO2 emissions: {total_co2_emissions:,.2f} kg CO2 ({total_co2_emissions_tons:,.2f} tons CO2)")
    
    # Print the direct answer for Q20
    print(f"\nQ20 Answer: The total CO2 emission from Ljusteröleden in February 2024 is {total_co2_emissions:,.2f} kg, assuming 2.68 kg CO2 per liter of fuel.")
    
    return total_co2_emissions

# Execute the function
calculate_ljustero_co2_emissions_feb2024(ferry_data)

CO2 Emissions Analysis for Ljusteröleden in February 2024:
Total fuel consumption: 16,359.66 liters
CO2 emission factor: 2.68 kg CO2 per liter
Total CO2 emissions: 43,843.88 kg CO2 (43.84 tons CO2)

Q20 Answer: The total CO2 emission from Ljusteröleden in February 2024 is 43,843.88 kg, assuming 2.68 kg CO2 per liter of fuel.


43843.88434754202